In [1]:
!gdown -q 1cFJABvYmdap4OQFoFWfaX_dfrKhMBg0N
!gdown -q 1BJWgkCA458-rNP5VJ9wXTSKbSkcADIQz
!unzip -q /content/ricedata.zip
!unzip -q '/content/Images.zip' -d '/content/Images'

In [ ]:
!pip install timm==0.6.5
!pip install fastai==2.7.0

In [1]:
import timm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from scipy.stats import gmean
import gc

In [2]:
from fastai.vision.all import *
set_seed(2022)

AttributeError: ignored

In [ ]:
# Paths
data_dir = '/content/ricedata'

tr_df = pd.read_csv(f'{data_dir}/Train.csv')
te_df = pd.read_csv(f'{data_dir}/Test.csv')
subs = pd.read_csv(f'{data_dir}/SampleSubmission.csv')


In [ ]:
# Get images paths
img_path = 'Images'
files = get_image_files(img_path)

# Show on image
img = PILImage.create(files[0])
print(img.size)
img.to_thumb(128)


In [ ]:
tr_rgb = tr_df[~tr_df.Image_id.str.contains('_rgn')].reset_index(drop=True) # Just the RGB images
tr_rgb[:3]

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
def train(train_ds, test_ds, backbone, item, batch, epochs):
    dls = ImageDataLoaders.from_df(train_ds, fn_col='Image_id', label_col='Label', seed= 2022, bs=16, valid_col="is_valid",
                                   folder=img_path, item_tfms=item, batch_tfms = batch)
    
    learn = vision_learner(dls, backbone, metrics=error_rate, opt_func=RAdam).to_fp16()

    callbacks = [SaveModelCallback(),
                 ReduceLROnPlateau(monitor='valid_loss', patience=3),
                ]
    
    learn.fine_tune(epochs, 0.01, cbs= callbacks)
    
    valid = learn.dls.valid
    val_preds, targets = learn.tta(dl=valid, n=8)
    
    te_preds, _ = learn.tta(dl=dls.test_dl(test_ds), n=8)
    
    logloss = log_loss(targets, val_preds) 
    
    return logloss, te_preds.numpy(), val_preds.numpy(), learn

In [ ]:
set_seed(2022,True)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2022)

te_dataset = te_df[~te_df.Image_id.str.contains('_rgn')] # Just the RGB images

train_preds = np.zeros((tr_rgb.shape[0],3)) 

preds = []
scores = []
i=0
for idx , (tr , val) in enumerate(skf.split(tr_rgb, tr_rgb['Label'])):
    print(f'########### Fold {i+1} / {skf.n_splits} ')
    tr_rgb['is_valid'] = False
    tr_rgb.at[val, 'is_valid'] = True

    score, te_preds, val_preds, learn = train(tr_rgb, te_dataset,'swinv2_large_window12_192_22k', item=Resize(400),
                                              batch=aug_transforms(size=192, min_scale=0.75), epochs=15)
    

    print(f'log loss: {score}')
    train_preds[val] = val_preds
    preds.append(te_preds)
    scores.append(score)
    
    labels = learn.dls.vocab
    del score, te_preds, val_preds, learn ; gc.collect()
    torch.cuda.empty_cache()
    i+=1
    
print(f'mean logloss: {np.mean(scores)}')

In [ ]:
final_test_predictions = gmean(preds, axis=0)
submission = pd.DataFrame({'Image_id': te_dataset['Image_id']})

for i, label in enumerate(labels):
    submission[label] = final_test_predictions[:,i] 
    
submission.to_csv('./subs.csv', index=False)